In [1]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade lxml

  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.0
    Uninstalling beautifulsoup4-4.9.0:
      Successfully uninstalled beautifulsoup4-4.9.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.5.0
    Uninstalling lxml-4.5.0:
      Successfully uninstalled lxml-4.5.0


In [2]:
from bs4 import BeautifulSoup,Comment

In [3]:
%run prepare_some_data.ipynb

Requirement already up-to-date: pymorphy2-dicts in c:\users\acer\anaconda3\lib\site-packages (2.4.393442.3710985)
Requirement already up-to-date: DAWG-Python in c:\users\acer\anaconda3\lib\site-packages (0.7.2)
Requirement already up-to-date: pymorphy2[fast] in c:\users\acer\anaconda3\lib\site-packages (0.8)
Requirement already up-to-date: nltk in c:\users\acer\anaconda3\lib\site-packages (3.5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


великий


In [4]:
def is_comment(element): 
        return isinstance(element, Comment)

In [5]:
def extract_for_bert(soup):
        
    blacklist = set([
        '[document]',
        
        'dialog',
        'html',
        'input',
        
        'script',
        'noscript',
        'applet',
        'embed',  
        'object',
        'param',
        
        
        'head',
        'meta',
        'base',
        'basefont',

        'style',
    ])
        
    for comment in soup.find_all(text=is_comment):
        comment.extract()
    
    
    ans=''
    for text in soup.find_all(text=True):
        if text.parent.name in blacklist:
            continue
        ans+=process_string(text)+' '
    
    return ans

In [6]:
def extract_for_bm(soup):
        
    blacklist = set([
        '[document]',
        
        'dialog',
        'html',
        'input',
        
        'script',
        'noscript',
        'applet',
        'embed',  
        'object',
        'param',
        
        
        'head',
        'meta',
        'base',
        'basefont',

        'style',
    ])
        
    for comment in soup.find_all(text=is_comment):
        comment.extract()
    
          
    classes=[
             set(['a']),
             set(['h1','h2']),
             set(['h3','h4','h5','h6']),
             set(['strong','b','em','i','u','dl','ol','ul'])
    ]
    ans=['' for i in range(len(classes)+1)]
    for text in soup.find_all(text=True):
        if text.parent.name in blacklist:
            continue
        ind=0
        for cls in classes:
            if text.parent.name in cls:
                ans[ind]+=string_lem(text)+' '
                break
            ind+=1
        if ind==len(classes):
            ans[ind]+=string_lem(text)+' '
    return reduce(lambda x,y:x+'\t'+y,ans)
    
    
    return ans

In [0]:
no_head=0
no_body=0

beg=time()


i=0
er=0
with ZipFile('data.zip') as myzip:
    files=myzip.namelist()
    with open('bert_docs.txt','w', errors = 'ignore') as out:
        for file_name in files:
            if file_name[-1] == '/':
                continue
            with myzip.open(file_name) as f:
                url=f.readline().decode('utf-8').strip()
                doc_id=get_id[url]
                raw_html=f.read()
                try:
                    html=raw_html.decode('utf-8')
                except UnicodeDecodeError:
                    html=raw_html.decode('latin-1')
                    er+=1
            
            out.write(str(doc_id)+'\t')
            soup=BeautifulSoup(html)
            if soup.head:
                tmp=soup.head.find('meta',{'name':'description'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'author'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'keywords'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.title
                out.write(process_string(tmp.text) if tmp is not None else '')
                out.write('\t')
            else:
                out.write('\t'*4)
                no_head+=1

            if soup.body:
                out.write(extract_for_bert(soup.body))
            else:
                no_body+=1
            out.write('\n')

            if i%100==0:
                print(i)
            i+=1

print(no_head,no_body,er)
print((time()-beg)/i)

In [18]:
no_head=0
no_body=0

beg=time()


i=0
er=0
with ZipFile('data.zip') as myzip:
    files=myzip.namelist()
    with open('bm_docs.txt','w', errors = 'ignore') as out:
        for file_name in files:
            if file_name[-1] == '/':
                continue
            with myzip.open(file_name) as f:
                url=f.readline().decode('utf-8').strip()
                doc_id=get_id[url]
                raw_html=f.read()
                try:
                    html=raw_html.decode('utf-8')
                except UnicodeDecodeError:
                    html=raw_html.decode('latin-1')
                    er+=1
            
            out.write(str(doc_id)+'\t')
            soup=BeautifulSoup(html)
            if soup.head:
                tmp=soup.head.find('meta',{'name':'description'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'author'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'keywords'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.title
                out.write(process_string(tmp.text) if tmp is not None else '')
                out.write('\t')
            else:
                out.write('\t'*4)
                no_head+=1

            if soup.body:
                out.write(extract_for_bm(soup.body))
            else:
                no_body+=1
                out.write('\t'*4)
            out.write('\n')

            if i%100==0:
                print(i)
            i+=1

print(no_head,no_body,er)
print((time()-beg)/i)

0
100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 